**SimPy** - фреймворк Python для дискретно-событийного моделирования систем.<br/>
В системах присутствуют *агенты*, которые подвергаются различным *процессам*. Например, в аэропорту пассажиры (агенты) проходят проверку безопасности (процесс), в колл-центре прцессом будет общение клиента с оператором. Все процессы происходят в некоторой *среде*. Среда также обладает некоторыми *ресурсами*.  Ресурсы – это части среды, количество которых ограничено, их использование требует определенного времени.

Общий принцип работы выглядит следующим образом:

In [ ]:
# Настройка среды
env = simpy.Environment() # создание объекта среды env, который управляет временем моделирования

# Функция checkpoint_run() описывает работу системы
env.process(checkpoint_run(env, num_booths, check_time, passenger_arrival)) # передаем переменные - параметры среды, которые можно изменять в процессе моделирования

# Запуск моделирования
env.run(until=10) # аргументом является требуемое время моделирования

Важным типом события является `Timeout`, который позволяют процессу находиться в спящем режиме, т.е. события этого типа запускаются по прошествии определенного времени.<br/>
Процессы в SimPy описываются функциями-генераторами.

### **Оператор yield**

`yield` используется в функциях для создания генераторов - объектов, которые генерируют значения "на лету", а не возвращают их все сразу.<br/>
Гераторы подходят для потоковой обработки данных, для работы с с большими или бесконечными последовательностями данных.

#### **Особенности yield:**
* Ленивые вычисления (Lazy Evaluation) - вычисления только когда нужно
* Сохранение состояния - функция "запоминает", где она остановилась
* Генератор одноразовый - после полного прохода нужно создавать заново

#### **Сравнение: return vs yield**
##### **return (обычная функция):**
```
def get_numbers():
    numbers = []
    for i in range(3):
        numbers.append(i)
    return numbers  # Возвращает ВСЕ числа сразу

result = get_numbers()
print(result)  # [0, 1, 2] - все данные в памяти
```
##### **yield (генератор):**
```
def generate_numbers():
    for i in range(3):
        yield i  # Возвращает по одному числу за раз

gen = generate_numbers()
print(next(gen))  # 0 - только первое число в памяти
print(next(gen))  # 1 - затем второе
print(next(gen))  # 2 - и третье
```




In [ ]:
!pip install simpy

Пример 1. **Моделирование работы светофора**

In [ ]:
# Simulation of a Traffic Light

# import the SimPy package
import simpy

# Generator function that defines the working of the traffic light
# "timeout()" заставляет следующий оператор yield ждать заданное время, переданное в качестве аргумента

# функция Traffic_Light(env) принимает переменную среды в качестве аргумента и имитирует работу светофора в течение периода времени, заданного в качестве аргумента в функции env.run()

def Traffic_Light(env):

	while True:

		print ("Light turns GRN at " + str(env.now)) # env.now возвращает текущее значение времени

		# Light is green for 25 seconds
		yield env.timeout(25)		# функция env.timeout() ожидает истечения заданного промежутка времени,
                            # затем инициирует следующий оператор yield, пока не истечет время, переданное в качестве аргумента в env.run()

		print ("Light turns YEL at " + str(env.now))

		# Light is yellow for 5 seconds
		yield env.timeout(5)

		print ("Light turns RED at " + str(env.now))

		# Light is red for 60 seconds
		yield env.timeout(60)

# env is the environment variable
env = simpy.Environment()

# The process defined by the function Traffic_Light(env)
# is added to the environment
env.process(Traffic_Light(env))

# The process is run for the first 180 seconds (180 is not included)
env.run(until = 180)


Light turns GRN at 0
Light turns YEL at 25
Light turns RED at 30
Light turns GRN at 90
Light turns YEL at 115
Light turns RED at 120


# Пример 2. Моделирование работы банковского отделения <br/>
Ресурсами являются сотрудники и терминалы по выдаче талонов. Требуется рассчитать среднее время обслуживание клиента.

In [ ]:
# импортируем необходимые библиотеки и создаем список времен
import random
import simpy
import statistics

times = []

# Для создания среды, создадим класс Банк, ресурсами в котором будут сотрудники банка и терминалы,
# и два процесса: выдача талона и обслуживание клиента
class Bank(object):
  def __init__(self, env, num_employee, num_terminal):
    self.env = env
    self.employee = simpy.Resource(env, num_employee)
    self.terminal = simpy.Resource(env, num_terminal)

  def service(self, customer):
	# Обслуживание занимает время в интервале от 1 до 15 минут
    yield self.env.timeout (random.randint(1, 15))

  def take_token(self, customer):
	# Получение талона занимает до 45 секунд
    yield self.env.timeout(45/60)

# функцию, отвечающая за поведение клиента в среде
def go_to_bank(env, customer, bank):
	# Клиент пришел в банк
	arrival_time = env.now

	with bank.terminal.request() as request:
		yield request
		yield env.process(bank.take_token(customer))

	with bank.employee.request() as request:
		yield request
		yield env.process(bank.service(customer))

	times.append(env.now-arrival_time)

# функция для запуска моделирования
# Она отвечает за создание экземпляра банка и генерацию клиентов до тех пор, пока симуляция не остановится
def run_bank(env, num_employee, num_terminal):
	bank = Bank(env, num_employee, num_terminal)

	customer = 1
	env.process(go_to_bank(env, customer, bank))
	while True:
		# Предположим каждые 4 минуты заходит новый клиент
		yield env.timeout(random.expovariate(1.0 / 4))

		customer += 1
		env.process(go_to_bank(env,customer, bank))

 # функция для подсчета среднего времени
def get_average_time(times):
	average_time = statistics.mean(times)

	minutes, frac_minutes = divmod(average_time, 1)

	seconds = frac_minutes * 60
	return round(minutes), round(seconds)

# Запуск моделирования
random.seed(42)
# Начальные данные
num_employee = 7 # В офисе работает 7 сотрудников
num_terminal = 1 # В офисе 1 терминал по выдаче талонов

# Запуск моделирования
env = simpy.Environment()
env.process(run_bank(env, num_employee, num_terminal))
env.run(until=12)

# Результаты
mins, secs = get_average_time(times)
print(f"\nСреднее время обслуживания: {mins}:{secs}")


Среднее время обслуживания: 6:57


# Пример 3. Моделирование работы кинотеатра

Целью моделирования является определить оптимальное количество сотрудников кинотеатра, чтобы среднее время ожидания каждого зрителя от момента прихода в кинотеатр до момента, когда он займет свое место, составляло не более 10 минут.

In [ ]:
"""Companion code to https://realpython.com/simulation-with-simpy/
'Simulating Real-World Processes With SimPy'
"""
# импортируем необходимые библиотеки
import simpy
import random
import statistics

# Создаем список для хранения времени, необходимого каждому зрителю, чтобы добраться до своего места
wait_times = []

# Для описания среды создаем класс Theater
# ресурсами являются кассиры, билетеры и киоски для продажи еды
# процессы: покупка билета, проверка билета, покупка еды
# Кассиры (cashiers) - это ресурс, который кинотеатр предоставляет своим клиентам, они помогают зрителям в процессе покупки билета.
# Ресурсами также являются билетеры (ushers) и киоски для продажи еды (servers)

class Theater(object):
    def __init__(self, env, num_cashiers, num_servers, num_ushers):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashiers)
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)

    def purchase_ticket(self, moviegoer): # покупка билета зрителем
        yield self.env.timeout(random.randint(1, 3)) # в среднем на оформление билета в кассе уходит от 1 до 3 минут

    def check_ticket(self, moviegoer): # проверка билета
        yield self.env.timeout(3 / 60) # билетеры проверяют билеты в среднем за 3 секунды

    def sell_food(self, moviegoer):
        yield self.env.timeout(random.randint(1, 5)) # покупка еды в среднем занимает от 1 до 5 минут


def go_to_movies(env, moviegoer, theater):
    # Кинозритель приходит в театр
    arrival_time = env.now # сохраняем время, когда зритель пришел в театр

# Запрос на использование ресурсов

# Касса является общим ресурсом, что означает, что многие зрители будут использовать одну и ту же кассу.
# Однако кассир может одновременно обслуживать только одного зрителя,
# поэтому требуется добавить некоторое поведение ожидания.

    with theater.cashier.request() as request: # зритель делает запрос к кассиру для покупки билета
        yield request # зритель ожидает, пока кассир освободится
        yield env.process(theater.purchase_ticket(moviegoer)) # покупка билета зрителем

   # Процесс проверки билета аналогичен
    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))

  # Поскольку неизвестно заранее, захочет ли зритель покупать еду,
  # вводится элемент случайности через модуль random и логические переменные True и False
    if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))

    # Зритель занимает свое место
    wait_times.append(env.now - arrival_time) # рассчитываем, сколько времени составило ожидание зрителя, и сохраняем его

# функция для запуска моделирования
def run_theater(env, num_cashiers, num_servers, num_ushers): # функция run_theater () отвечает за создание экземпляра кинотеатра и генерацию зрителей, пока симуляция не прекратится
    theater = Theater(env, num_cashiers, num_servers, num_ushers) #  настройка кинотеатра с определенным количеством кассиров, киосков для еды и билетеров

    for moviegoer in range(3): # считаем, что первоначально в кинотеатре три человека уже ожидают обслуживания
        env.process(go_to_movies(env, moviegoer, theater))

    while True:
        yield env.timeout(0.20)  # зрители приходят в кинотеатр в среднем каждые 12 секунд
                                 # делаем на это время задержку, прежде чем создавать нового пользователя

        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, theater))

# функция для рассчета среднего времени ожидания
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)

    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

# Чтобы иметь возможность изменять параметры в процессе моделирования, создаем для них поля ввода
def get_user_input():
    num_cashiers = input("Input # of cashiers working: ")
    num_servers = input("Input # of servers working: ")
    num_ushers = input("Input # of ushers working: ")
    params = [num_cashiers, num_servers, num_ushers]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Не удалось проанализировать входные данные. При моделировании будут использоваться значения по умолчанию:",
            "\n1 cashier, 1 server, 1 usher.",
        )
        params = [1, 1, 1]
    return params


def main():
    # Настройка среды
    random.seed(42)
    num_cashiers, num_servers, num_ushers = get_user_input()

    # Запуск моделирования
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
    env.run(until=90)

    # Просмотр результатов
    mins, secs = get_average_wait_time(wait_times)
    print(
        "Моделирование запущено...",
        f"\nСреднее время ожидания {mins} минут и {secs} секунд.",
    )


if __name__ == "__main__":
    main()


Input # of cashiers working: 10
Input # of servers working: 10
Input # of ushers working: 5
Моделирование запущено... 
Среднее время ожидания 4 минут и 33 секунд.


Больше примеров см. в [документации](https://simpy.readthedocs.io/en/latest/examples/index.html)